In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

# Fetch My Health Data From Appple
> I've exported all my appleHealth data from the app in xml Format and I'm going to read all the data into pandas dataframe

In [2]:
# create element tree object
tree = ET.parse('/Users/emretuygan/Desktop/apple_health_export/appleData1.xml') 
# for every health record, extract the attributes
root = tree.getroot()
record_list = [x.attrib for x in root.iter('Record')]

In [3]:
record_data = pd.DataFrame(record_list)
# proper type to dates
for col in ['creationDate', 'startDate', 'endDate']:
    record_data[col] = pd.to_datetime(record_data[col])

# value is numeric, NaN if fails
record_data['value'] = pd.to_numeric(record_data['value'], errors='coerce')

# some records do not measure anything, just count occurences
# filling with 1.0 (= one time) makes it easier to aggregate
record_data['value'] = record_data['value'].fillna(1.0)

# shorter observation names
record_data['type'] = record_data['type'].str.replace('HKQuantityTypeIdentifier', '')
record_data['type'] = record_data['type'].str.replace('HKCategoryTypeIdentifier', '')

## Data that I'm interested in:
- 'RestingHeartRate'
- 'SleepAnalysis'
- 'StepCount'
- 'HKDataTypeSleepDurationGoal''
- 'HeartRateVariabilitySDNN'
- 'HeartRate'
- 'DietaryCaffeine'

# Resting Heart Rate - RHRdf
- startDate and endDate define an interval of the measurment
- value is my resting hearth rate
- provided unit is count/min
- indexes are the date of the RHR

In [4]:
# Filter Resting Heart Rate records and create a DataFrame
resting_heart_rate_data = record_data[record_data['type'] == 'RestingHeartRate']

# Create DataFrame and set 'creationDate' as index
RHRdf = pd.DataFrame(resting_heart_rate_data)
RHRdf.set_index('creationDate', inplace=True)

# Drop specified columns
columns_to_drop = ['type', 'unit', 'index', 'sourceName', 'sourceVersion', 'device']
RHRdf.drop(columns=columns_to_drop, inplace=True, errors='ignore')
RHRdf.head()

,startDate,endDate,value
creationDate,,,
2023-10-24 17:50:59+03:00,2023-10-24 00:04:39+03:00,2023-10-24 17:45:23+03:00,67.0
2023-10-25 22:23:11+03:00,2023-10-25 00:02:47+03:00,2023-10-25 22:19:09+03:00,61.0
2023-10-26 15:38:17+03:00,2023-10-26 00:06:30+03:00,2023-10-26 15:33:49+03:00,73.0
2023-10-29 22:34:46+03:00,2023-10-29 00:02:27+03:00,2023-10-29 16:48:04+03:00,66.0
2023-10-29 22:34:46+03:00,2023-10-28 00:47:17+03:00,2023-10-28 16:54:04+03:00,79.0


# Sleep Analysis df - sleep_df
- indexes are the creation dates
- bedTime is the start time of my sleep sequence
- awake time is the time when I wake up
- time_in_bed represent the time that I stay in bed

> Apple watch tracks your movement to find your sleep snippets so the provided data has bunch of sleep sessions for each day
- sleep_counts is how many times that I wake and sleep during night.

- total_time_asleep is the sum of durations that I'm actually sleeping in the night

I'm going to find total_time_asleep by evaluating the time differences between each sleep session


In [5]:
sleep_data = record_data.copy()

# Filter for SleepAnalysis records from the specific source
sleep_data = sleep_data[(sleep_data['type'] == 'SleepAnalysis') & 
                        (sleep_data['sourceName'] == "Emre Apple\xa0Watch’u")]

In [6]:
#handle duplicate rows
sleep_data = sleep_data.drop_duplicates(subset=['startDate', 'endDate'], keep='first')
#handle overlaps
# Identify overlapping records with the same start point
overlap_mask = sleep_data.duplicated(subset=['startDate'], keep=False)
# Calculate duration for each row
sleep_data['duration'] = sleep_data['endDate'] - sleep_data['startDate']
# Select rows with the minimum duration for each unique start point
min_duration_rows = sleep_data[overlap_mask].groupby('startDate')['duration'].idxmin()
# Keep the rows with the minimum duration and remove others
sleep_data = sleep_data.drop(index=sleep_data[overlap_mask].index.difference(min_duration_rows))

In [7]:
# calulate time between date(s)
sleep_data['time_asleep'] = sleep_data['endDate'] - sleep_data['startDate']

sleep_data = sleep_data.groupby('creationDate').agg(
    total_time_asleep=('time_asleep', 'sum'),
    bed_time=('startDate', 'min'), 
    awake_time=('endDate', 'max'), 
    sleep_counts=('creationDate','count'))

sleep_data['time_in_bed'] = sleep_data['awake_time'] - sleep_data['bed_time']
sleep_data['bed_time'] = pd.to_datetime(sleep_data['bed_time'])
sleep_data['bed_time'] = sleep_data['bed_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
sleep_data['awake_time'] = pd.to_datetime(sleep_data['awake_time'])
sleep_data['awake_time'] = sleep_data['awake_time'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Function to format timedelta to hours:minutes:seconds
def format_timedelta(td):
    hours, remainder = divmod(td.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f'{hours:02}:{minutes:02}:{seconds:02}'

sleep_data['time_in_bed'] = sleep_data['time_in_bed'].apply(format_timedelta)
sleep_data['total_time_asleep'] = sleep_data['total_time_asleep'].apply(format_timedelta)

sleep_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 42 entries, 2023-10-24 08:11:14+03:00 to 2023-12-11 10:14:37+03:00
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   total_time_asleep  42 non-null     object
 1   bed_time           42 non-null     object
 2   awake_time         42 non-null     object
 3   sleep_counts       42 non-null     int64 
 4   time_in_bed        42 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.0+ KB


# My Caffeine Consumption Data: - caffein_df
> I've been logging my caffeine consumption in an app called HiCoffe. It is logging the caffeine consumption to apple health as well bu I'm going to get my directly from the app as it is providing my caffein consumption in hourly basis (apple only provides daily).

In [10]:
# Replace 'file_path.csv' with your CSV file's path
file_path = '/Users/emretuygan/Desktop/HiCoffee_Exported-Data_UTF8_2023-12-11_11-44-15.csv'

# Read the CSV file into a DataFrame
HiCoffe_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to verify the data has been loaded
HiCoffe_df.head()

,Date,Beverage,Brand,Caffeine (mg)
0,"Oct 21, 2023 at 16:30:44",Espresso,-,75
1,"Oct 22, 2023 at 13:55:15",Coffee Capsule,-,60
2,"Oct 22, 2023 at 16:55:27",Diet Coke (250 mL),-,32
3,"Oct 22, 2023 at 18:54:52",Espresso,-,75
4,"Oct 23, 2023 at 9:56:00",Brewed Coffee - Decaf Pike Place Roast (Short),Starbucks,15


## Edit my consumtion data:
- I want to cluster my consumption in daily basis.
- I'm going to divide my day into four and structure my consumption in those four timezones which are ['08.00-12.00', '12.00-16.00', '16.00-20.00', '20.00-24.00']
By that I can also investigate the affect of consuming coffe on significant interaval ass well as my total consumption

In [12]:
caffein_df = HiCoffe_df.copy()

# Convert 'Date' column to datetime
caffein_df['Date'] = pd.to_datetime(caffein_df['Date'], format='%b %d, %Y at %H:%M:%S')

# Create time interval columns
caffein_df['Caffeine 08.00-12.00'] = 0
caffein_df['Caffeine 12.00-16.00'] = 0
caffein_df['Caffeine 16.00-20.00'] = 0
caffein_df['Caffeine 20.00-24.00'] = 0

# Assign caffeine consumption to time interval columns
for index, row in caffein_df.iterrows():
    hour = row['Date'].hour
    caffeine = row['Caffeine (mg)']
    
    if 8 <= hour < 12:
        caffein_df.at[index, 'Caffeine 08.00-12.00'] = caffeine
    elif 12 <= hour < 16:
        caffein_df.at[index, 'Caffeine 12.00-16.00'] = caffeine
    elif 16 <= hour < 20:
        caffein_df.at[index, 'Caffeine 16.00-20.00'] = caffeine
    elif 20 <= hour <= 23:
        caffein_df.at[index, 'Caffeine 20.00-24.00'] = caffeine

# Group by Date and sum up the columns
caffein_df = caffein_df.groupby(caffein_df['Date'].dt.date).sum()
# Reset index to make 'Date' a column again
caffein_df.reset_index(inplace=True)
# Set 'Date' as index
caffein_df.set_index('Date', inplace=True)

caffein_df

,Caffeine (mg),Caffeine 08.00-12.00,Caffeine 12.00-16.00,Caffeine 16.00-20.00,Caffeine 20.00-24.00
Date,,,,,
2023-10-21,75,0,0,75,0
2023-10-22,167,0,60,107,0
2023-10-23,81,15,66,0,0
2023-10-24,88,46,42,0,0
2023-10-25,34,2,32,0,0
2023-10-26,85,53,32,0,0
2023-10-27,150,0,0,150,0
2023-10-28,85,85,0,0,0
2023-10-29,299,42,225,32,0
